In [1]:
import pymc as pm
import numpy as np
import arviz as az
from pymc.math import switch, ge, exp

%load_ext lab_black
%load_ext watermark

# Revisiting UK coal mining disasters

This example demonstrates ...

Adapted from [unit 10: disasters.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/disasters.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

Associated lecture video: Unit 10 Lesson 3

## Problem statement

Change Point Analysis, discussed in Unit 5 (Gibbs Sampler).

British coal mine disaster data by year (1851-1962)  

The 112 data points represent the numbers of coal-mining disasters involving 10 or more men killed per year between  1851 and 1962. 
 
Based on the observation that the there was a significant decrease around 1900, it is suitable to apply a change-point model to divide the whole dataset into two periods; each period with its own distribution of number of disasters.
 
The data set was compiled by Maguire, Pearson and Wynn in 1952 and 
updated by Jarrett (1978). This data have been used by a number of authors to illustrate various techniques that can be applied to point processes


 Maguire, B. A., Pearson, E. S. and Wynn, A. H. A. (1952). The time intervals between industrial accidents.   Biometrika, 39, 168†180.

 Jarrett, R.G. (1979). A note on the intervals between coal-mining disasters. Biometrika, 66, 191-193. 

 Carlin, Gelfand, and Smith (1992) Heirarchical Bayesian Analysis of Changepoint Problems. Applied Statistics, 41, 389-405.


In [3]:
# X is the number of coal mine disasters per year
# fmt: off
X = np.array([4, 5, 4, 1, 0, 4, 3, 4, 0, 6, 3, 3, 4, 0, 2, 6, 3, 3, 5, 4, 5, 3, 1,
     4, 4, 1, 5, 5, 3, 4, 2, 5, 2, 2, 3, 4, 2, 1, 3, 2, 2, 1, 1, 1, 1, 3,
     0, 0, 1, 0, 1, 1, 0, 0, 3, 1, 0, 3, 2, 2, 0, 1, 1, 1, 0, 1, 0, 1, 0,
     0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2, 3, 3, 1, 1, 2, 1, 1, 1, 1, 2, 4, 2,
     0, 0, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])
# fmt: on

y = np.array([y for y in range(1851, 1963)])

## Model 1

In [4]:
α = 4
β = 1
γ = 0.5
δ = 1

with pm.Model() as m:
    year = pm.Uniform("year", 1851, 1963)
    λ = pm.Gamma("λ", α, β)
    μ = pm.Gamma("μ", γ, δ)

    diff = pm.Deterministic("diff", μ - λ)

    rate = λ + switch(ge(y - year, 0), 1, 0) * diff
    pm.Poisson("lik", mu=rate, observed=X)

    trace = pm.sample(2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [year, λ, μ]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 24 seconds.


In [5]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
year,1890.402,2.458,1886.003,1894.696,0.047,0.033,2820.0,2965.0,1.0
λ,3.146,0.291,2.591,3.691,0.005,0.004,2848.0,3453.0,1.0
μ,0.920,0.114,0.708,1.138,0.002,0.002,2717.0,2660.0,1.0
diff,-2.226,0.306,-2.835,-1.670,0.006,0.004,3046.0,3290.0,1.0


## Model 2

In [6]:
with pm.Model() as m:
    year = pm.Uniform("year", 1851, 1963)
    z0 = pm.Normal("z0", 0, tau=0.00001)
    z1 = pm.Normal("z1", 0, tau=0.00001)

    λ = pm.Deterministic("λ", exp(z0))
    μ = pm.Deterministic("μ", exp(z0 + z1))

    diff = pm.Deterministic("diff", μ - λ)

    rate = pm.math.exp(z0 + switch(ge(y - year, 0), 1, 0) * z1)
    pm.Poisson("lik", mu=rate, observed=X)

    trace = pm.sample(2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [year, z0, z1]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 30 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [7]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
z0,1.134,0.093,0.961,1.310,0.002,0.001,2093.0,2977.0,1.0
z1,-1.225,0.152,-1.511,-0.942,0.003,0.002,2202.0,2866.0,1.0
year,1890.477,2.464,1885.999,1894.862,0.047,0.033,2779.0,2891.0,1.0
λ,3.123,0.292,2.598,3.682,0.006,0.005,2093.0,2977.0,1.0
μ,0.920,0.118,0.706,1.142,0.002,0.001,3484.0,3758.0,1.0
diff,-2.202,0.304,-2.769,-1.643,0.007,0.005,1900.0,2628.0,1.0


In [8]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

numpy: 1.22.4
arviz: 0.12.1
pymc : 4.0.0

